<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/jangjaewon/20221022_code_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta


from tqdm.autonotebook import tqdm
import csv

import re

In [ ]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [ ]:
prescriptions = pd.read_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/prescriptions_ab.csv', low_memory=False, dtype='str', keep_default_na=False)

In [ ]:
prescriptions.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'starttime', 'stoptime', 'drug_type', 'drug', 'formulary_drug_cd',
       'gsn', 'ndc', 'prod_strength', 'form_rx', 'dose_val_rx', 'dose_unit_rx',
       'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route',
       'sepsis3', 'intime', 'outtime'],
      dtype='object')

In [ ]:
icustays.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los'],
      dtype='object')

In [ ]:
icustays = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/icu/icustays.csv', low_memory=False, dtype='str', keep_default_na=False)
prescriptions = pd.merge(prescriptions, icustays[['hadm_id','intime','outtime']], left_on='hadm_id', right_on='hadm_id', how='left')

In [ ]:
# sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)
# prescriptions = pd.merge(left = prescriptions, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')

#prescription 첫줄만 가져오도록

In [ ]:
prescriptions.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'starttime', 'stoptime', 'drug_type', 'drug', 'formulary_drug_cd',
       'gsn', 'ndc', 'prod_strength', 'form_rx', 'dose_val_rx', 'dose_unit_rx',
       'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route'],
      dtype='object')

In [ ]:
prescriptions.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22841357,53555857,10000032-94,94.0,2180-06-26 23:00:00,2180-06-27 23:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,904272561.0,DS Tablet,,1,TAB,1,TAB,1.0,PO/NG
1,10000117,27988844,52240876,10000117-119,119.0,2183-09-19 12:00:00,2183-09-20 15:00:00,MAIN,cefazolin,CEFA2D100,075120,338350841.0,2 g / 100 mL Dextrose (iso-os),,2,g,1,BAG,3.0,IV
2,10000724,20823482,5703283,10000724-101,101.0,2145-04-25 00:00:00,2145-04-28 20:00:00,MAIN,cefazolin,CEFA1F,051880,264310311.0,1g Froz.Bag,,1,g,1,BAG,3.0,IV
3,10000764,27897940,67322733,10000764-50,50.0,2132-10-15 20:00:00,2132-10-19 00:00:00,MAIN,amoxicillin-clavulanic acid,AUGM500,008992,66685100202.0,500mg Tablet,,500,mg,1,TAB,3.0,PO/NG
4,10000826,20032235,52793571,10000826-121,121.0,2146-12-09 10:00:00,2146-12-12 13:00:00,MAIN,ciprofloxacin hcl,CIPR500,009510,172531210.0,500mg Tab,,500,mg,1,TAB,2.0,PO/NG


In [ ]:
prescriptions['drug']

0                       azithromycin
1                       azithromycin
2                       azithromycin
3          sulfameth/trimethoprim ss
4                       levofloxacin
                     ...            
686942        metronidazole (flagyl)
686943                  levofloxacin
686944                    vancomycin
686945    piperacillin-tazobactam na
686946     sulfameth/trimethoprim ds
Name: drug, Length: 686947, dtype: object

In [ ]:
EXTRACT_FEATURE_CODES_ANTI_2 = dict({
   '225850': 'Cephalosporin', '225851': 'Cephalosporin', '225853': 'Cephalosporin', '225855': 'Cephalosporin', '229587':'Cephalosporin',
   '225798': 'Glycopetide',
   '225842': 'Penicillin', '225843': 'Penicillin', '225888': 'Penicillin', '225889': 'Penicillin', '225890': 'Penicillin', '225892': 'Penicillin', '225893': 'Penicillin',
   '225859': 'Quninolone', '225859': 'Quninolone', '225879': 'Quninolone', '225886': 'Quninolone',
   '225845': 'Macrolide', '225865': 'Macrolide', '225866': 'Macrolide'
})

In [ ]:
EXTRACT_FEATURE_CODES_ANTI_2_list = list(EXTRACT_FEATURE_CODES_ANTI_2.keys())

In [ ]:
# 항생제 레이블링하기
anti_labels = dict()
for row in tqdm(prescriptions, desc='Get anti Label'):
  hadm_id = row['hadm_id']
  if row['drug'] in EXTRACT_FEATURE_CODES_ANTI_2_list:
    anti_label_group = row['drug'][0]
    anti_label_time = row['starttime'][0]
  else:
    anti_label_group = 'none'
    anti_label_time = 'none'
  anti_labels[hadm_id] = dict({'anti': anti_label_group, 'anti_TIME': anti_label_time})

Get anti Label:   0%|          | 0/686947 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
prescriptions['hadm_id']

0         23594368
1         24746267
2         24746267
3         24746267
4         24746267
            ...   
686942    21033226
686943    21033226
686944    21033226
686945    21033226
686946    26071774
Name: hadm_id, Length: 686947, dtype: object

In [ ]:
str(prescriptions['hadm_id'])

'0         23594368\n1         24746267\n2         24746267\n3         24746267\n4         24746267\n            ...   \n686942    21033226\n686943    21033226\n686944    21033226\n686945    21033226\n686946    26071774\nName: hadm_id, Length: 686947, dtype: object'